In [8]:
# prerequisites
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from torchvision.utils import save_image

bs = 100
# MNIST Dataset
train_dataset = datasets.FashionMNIST(root='./fashion_mnist_data/', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.FashionMNIST(root='./fashion_mnist_data/', train=False, transform=transforms.ToTensor(), download=False)
# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=bs, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=bs, shuffle=False)
# print(train_loader.dataset)

100%|██████████| 26421880/26421880 [00:02<00:00, 9245264.63it/s] 


Extracting ./fashion_mnist_data/FashionMNIST\raw\train-images-idx3-ubyte.gz to ./fashion_mnist_data/FashionMNIST\raw



100%|██████████| 29515/29515 [00:00<00:00, 180940.64it/s]


Extracting ./fashion_mnist_data/FashionMNIST\raw\train-labels-idx1-ubyte.gz to ./fashion_mnist_data/FashionMNIST\raw



100%|██████████| 4422102/4422102 [00:01<00:00, 3023166.94it/s]


Extracting ./fashion_mnist_data/FashionMNIST\raw\t10k-images-idx3-ubyte.gz to ./fashion_mnist_data/FashionMNIST\raw



100%|██████████| 5148/5148 [00:00<00:00, 15545195.82it/s]

Extracting ./fashion_mnist_data/FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to ./fashion_mnist_data/FashionMNIST\raw



In [9]:
class VAE(nn.Module):
    def __init__(self, x_dim, h_dim1, h_dim2, z_dim):
        super(VAE, self).__init__()
        
        # encoder part
        self.fc1 = nn.Linear(x_dim, h_dim1)
        self.fc2 = nn.Linear(h_dim1, h_dim2)
        self.fc31 = nn.Linear(h_dim2, z_dim)
        self.fc32 = nn.Linear(h_dim2, z_dim)
        # decoder part
        self.fc4 = nn.Linear(z_dim, h_dim2)
        self.fc5 = nn.Linear(h_dim2, h_dim1)
        self.fc6 = nn.Linear(h_dim1, x_dim)
        
    def encoder(self, x):
        h = F.relu(self.fc1(x))
        h = F.relu(self.fc2(h))
        return self.fc31(h), self.fc32(h) # mu, log_var
    
    def sampling(self, mu, log_var):
        std = torch.exp(0.5*log_var)
        eps = torch.randn_like(std)
        return eps.mul(std).add_(mu) # return z sample
        
    def decoder(self, z):
        h = F.relu(self.fc4(z))
        h = F.relu(self.fc5(h))
        return F.sigmoid(self.fc6(h)) 
    
    def forward(self, x):
        mu, log_var = self.encoder(x.view(-1, 784))
        z = self.sampling(mu, log_var)
        return self.decoder(z), mu, log_var

# build model
vae = VAE(x_dim=784, h_dim1= 512, h_dim2=256, z_dim=16)
if torch.cuda.is_available():
    vae.cuda()

In [10]:
vae

VAE(
  (fc1): Linear(in_features=784, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc31): Linear(in_features=256, out_features=16, bias=True)
  (fc32): Linear(in_features=256, out_features=16, bias=True)
  (fc4): Linear(in_features=16, out_features=256, bias=True)
  (fc5): Linear(in_features=256, out_features=512, bias=True)
  (fc6): Linear(in_features=512, out_features=784, bias=True)
)

In [11]:
optimizer = optim.Adam(vae.parameters())
# return reconstruction error + KL divergence losses
def loss_function(recon_x, x, mu, log_var):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='sum')
    KLD = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
    return BCE + KLD

In [12]:
def train(epoch):
    vae.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.cuda()
        optimizer.zero_grad()
        
        recon_batch, mu, log_var = vae(data)
        # print(log_var)
        loss = loss_function(recon_batch, data, mu, log_var)
        
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item() / len(data)))
    print('====> Epoch: {} Average loss: {:.4f}'.format(epoch, train_loss / len(train_loader.dataset)))

In [13]:
def test():
    vae.eval()
    test_loss= 0
    with torch.no_grad():
        for data, _ in test_loader:
            data = data.cuda()
            recon, mu, log_var = vae(data)
            
            # sum up batch loss
            test_loss += loss_function(recon, data, mu, log_var).item()
        
    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

In [14]:
for epoch in range(1, 51):
    train(epoch)
    test()

Train Epoch: 1 [0/60000 (0%)]	Loss: 544.669062
Train Epoch: 1 [10000/60000 (17%)]	Loss: 297.921602
Train Epoch: 1 [20000/60000 (33%)]	Loss: 292.657441
Train Epoch: 1 [30000/60000 (50%)]	Loss: 271.069531
Train Epoch: 1 [40000/60000 (67%)]	Loss: 262.748730
Train Epoch: 1 [50000/60000 (83%)]	Loss: 271.893242
====> Epoch: 1 Average loss: 282.0136
====> Test set loss: 257.1566
Train Epoch: 2 [0/60000 (0%)]	Loss: 258.492051
Train Epoch: 2 [10000/60000 (17%)]	Loss: 261.845078
Train Epoch: 2 [20000/60000 (33%)]	Loss: 246.383926
Train Epoch: 2 [30000/60000 (50%)]	Loss: 250.709141
Train Epoch: 2 [40000/60000 (67%)]	Loss: 245.744863
Train Epoch: 2 [50000/60000 (83%)]	Loss: 251.566992
====> Epoch: 2 Average loss: 251.3779
====> Test set loss: 249.9692
Train Epoch: 3 [0/60000 (0%)]	Loss: 250.923594
Train Epoch: 3 [10000/60000 (17%)]	Loss: 251.581152
Train Epoch: 3 [20000/60000 (33%)]	Loss: 252.616094
Train Epoch: 3 [30000/60000 (50%)]	Loss: 245.793086
Train Epoch: 3 [40000/60000 (67%)]	Loss: 246.63

In [15]:
with torch.no_grad():
    z = torch.randn(64, 16).cuda()
    sample = vae.decoder(z).cuda()
    print(z)
    save_image(sample.view(64, 1, 28, 28), './samples/sample_' + '.png')

tensor([[-0.0978, -0.7176,  0.0683,  ...,  1.4065,  1.1016,  0.3603],
        [-0.1261,  0.7596,  1.0101,  ..., -0.3925, -0.3763,  0.2948],
        [ 1.0834, -1.0406,  0.0337,  ...,  0.7026, -0.3118, -1.4886],
        ...,
        [ 1.5027,  0.6572, -0.5446,  ...,  0.7953,  0.9310,  0.1258],
        [ 0.0737,  0.1843, -0.1556,  ...,  0.5784,  1.4852, -0.6610],
        [ 0.0829,  1.3740,  1.0374,  ...,  0.3810, -0.6387, -0.0261]],
       device='cuda:0')


In [16]:
torch.save(vae, 'vaefashinon16.pth')